# Generating SLC stacks with topsStack

**Authors**: Bryan Riel

In this notebook, we will briefly demonstrate how to use `topsStack` to generate a stack of co-registered Sentinel-1 SLCs. The processing setup is identical to normal `topsApp.py` processing in that we download the appropriate SLC data, prepare a DEM for our study area, and download the necessary orbit and aux files from ASF/ESA (see previous tutorials on processing in TOPS mode with `topsApp.py`). More info on preparing for stacks can be found [here](https://github.com/parosen/Geo-SInC/tree/main/EarthScope2023/5.4_Intro_to_preparing_data_for_stack_processing).

<br>
<div class="alert alert-danger">
<font size="4"> <b> <font color='rgba(200,0,0,0.2)'> DISCLAIMER:  </font> </b> </font>

The following notebook has been run offline and is for demonstration purposes only. In order to run this notebook for your own data, you will need to download SLCs into an `asf` directory, place all orbits in an `orbits` directory, and all other auxiliary files in the `aux` directory. </i></b>
</div>

## Study area and setup

In this tutorial, our study area is Pine Island Glacier (PIG) in West Antartica. PIG is a fast-flowing ice stream and is responsible for about 25% of the mass loss from Antarctica for the past few decades. Here, we will use Sentinel-1 A/B IW SLCs for path 65, frame 906 (ascending) for two dates: 2020-01-12 and 2020-01-18. Moreover, we will only use one swath (Swath 2), which contains most of the fast-flowing regions of PIG (see image in main dense offsets notebook).

For the digital elevation model (DEM), we will use the Reference Elevation Model of Antarctica (REMA) 100 meter mosaic, which can be accessed here (https://www.pgc.umn.edu/data/rema/). We warp the REMA data from Polar Stereographic South to WGS84 and apply a geoid correction in order to use for ISCE processing. 

Let's first import necessary Python packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import datetime
import glob
import sys
import os
from osgeo import gdal, osr
import scipy.ndimage as ndimage
import isce

# Set environment variables for ISCE
isce_path = os.path.join(isce.__path__[0].strip(), 'applications')
os.environ['PATH'] = f"{isce_path}:{os.environ['PATH']}"

# setup path for topsStack
# $ISCE_STACK is set by conda
sys.path.insert(0, os.environ['ISCE_STACK'])
stack_path = os.path.join(os.environ['ISCE_STACK'], 'topsStack')
os.environ['PATH'] = f"{stack_path}:{os.environ['PATH']}"
os.environ['PYTHONPATH'] = f"{os.environ['ISCE_STACK']}:{os.environ['PYTHONPATH']}"

os.environ['OMP_NUM_THREADS'] = '8'
plt.rc('font', size=13)
parent_dir = os.getcwd()

Let's explore our top-level directory structure. Here, the S1A/B SLC zip files have been downloaded into the `asf` directory, and the orbit and auxiliary data have been downloaded into the `orbits` and `aux` directories, respectively.

In [ ]:
!ls asf
print('')
!ls orbits | tail
print('')
!ls aux

### Running stackSentinel.py

We use `stackSentinel.py` to generate all configuration and run files required to be executed for a stack of Sentinel-1 TOPS data. Let's first make sure we can find it in our path:

In [ ]:
!which stackSentinel.py

We now point `stackSentinel.py` to our input data and relevant processing options:
- `-W offset`: prepare files for generating dense offsets for various date pairs
- `-c 2`: generate a max of 2 offset pairs between each date and subsequent date
- `--swath_num '2'`: process swath 2
- `-b': specify the bounding box
- `-p hh`: specify HH polarization

In [ ]:
%%bash

(stackSentinel.py \
    -s asf \
    -o orbits \
     -a aux \
    -d pig_rema_100m_filled_wgs84.dem \
    -W offset \
    -c 2 \
    --swath_num '2' \
    -b '-75.6 -74.4 -104 -96' \
    -p hh \
    --num_proc 4 \
    --num_proc4topo 1)

In [ ]:
# Make sure run files are executable
!chmod a+x run_files/run_*

### Stack processing steps

We'll now go through each step of the processing by calling the run files one-by-one.

In [ ]:
# Run unpack and 'topo' step: this maps the DEM into the reference image geometry at burst overlaps.
# This data is used for the co-registration of secondary images into the reference geometry
os.environ['OMP_NUM_THREADS'] = '8'
subprocess.run('./run_files/run_01_unpack_topo_reference', shell=True)

In [ ]:
# This unpacks all of the secondary SLC metadata
subprocess.run('./run_files/run_02_unpack_secondary_slc', shell=True)

In [ ]:
# Compute the average baseline between the secondary and reference orbits
subprocess.run('./run_files/run_03_average_baseline', shell=True)

In [ ]:
# Extract SLC data over burst overlaps
subprocess.run('./run_files/run_04_extract_burst_overlaps', shell=True)

In [ ]:
# Compute mapping from DEM (geographic) coordinates to radar burst coordinates for each secondary SLC
os.environ['OMP_NUM_THREADS'] = '8'
subprocess.run('./run_files/run_05_overlap_geo2rdr', shell=True)

In [ ]:
# Resample secondary bursts to reference geometry
subprocess.run('./run_files/run_06_overlap_resample', shell=True)

In [ ]:
# Compute residual offsets between reference and secondary bursts over the overlaps
subprocess.run('./run_files/run_07_pairs_misreg', shell=True)

In [ ]:
# Compute time series of offsets for temporal consistency
subprocess.run('./run_files/run_08_timeseries_misreg', shell=True)

In [ ]:
# Compute mapping from DEM to radar coordinates for the full bursts for the secondary SLCs
os.environ['OMP_NUM_THREADS'] = '8'
subprocess.run('./run_files/run_09_fullBurst_geo2rdr', shell=True)

In [ ]:
# Resample secondary SLC bursts to reference
os.environ['OMP_NUM_THREADS'] = '8'
subprocess.run('./run_files/run_10_fullBurst_resample', shell=True)

In [ ]:
# Check the valid bursts (bursts that exist over all images)
subprocess.run('./run_files/run_11_extract_stack_valid_region', shell=True)

In [ ]:
# Compute fully merged SLC images
# This is the last step before dense offsets
subprocess.run('./run_files/run_12_merge_reference_secondary_slc', shell=True)

In [ ]:
!ls -l merged/SLC/*

In [ ]:
!ls -l merged/geom_reference

We now have all the files necessary to generate dense offsets. Unlike `topsApp.py`, the `run_13_dense_offsets` step only generates the offsets in radar coordinates and does not perform any filtering or geocoding. Please see the main dense offsets notebook for a complete processing workflow.